In [2]:
from synnet.utils.reconstruct_utils import *

15:50:35 rdkit INFO: Enabling RDKit 2022.09.5 jupyter extensions
15:51:02 faiss.loader INFO: Loading faiss with AVX2 support.
15:51:02 faiss.loader INFO: Successfully loaded faiss with AVX2 support.


In [3]:
import os
os.chdir('SynthesisNet/')
from argparse import Namespace

class Namespace:
    def __init__(self, **kwargs):
        self.__dict__.update(kwargs)

args = Namespace(
    rxns_collection_file='data/assets/reaction-templates/reactions_hb.json.gz',
    rxn_templates_file='data/assets/reaction-templates/hb.txt',
    building_blocks_file='data/assets/building-blocks/enamine_us_matched.csv',
    embeddings_knn_file='data/assets/building-blocks/enamine_us_emb_fp_256.npy',
)
load_data(args)

In [4]:
from synnet.utils.reconstruct_utils import rxns
from synnet.utils.data_utils import Reaction, fp_2048
from synnet.encoding.distances import _tanimoto_similarity

In [5]:
import pickle
skeletons = pickle.load(open('results/viz/top_1000/skeletons-top-1000.pkl', 'rb'))


In [6]:
def sanity_check():
    bad_rxn_ids = set()
    for index in range(len(skeletons)):
        key = list(skeletons)[index]        
        for st in skeletons[key]:
            sk = Skeleton(st, index)
            for n in np.argwhere(~sk.rxns & ~sk.leaves).flatten():             
                r = list(sk.tree.successors(n))[0]
                rxn_id = sk.tree.nodes[r]['rxn_id']
                rxn = rxns[rxn_id]
                smirks = rxn.smirks
                smiles = sk.tree.nodes[sk.tree_root]['smiles']
                retro_smirks = '>>'.join(smirks.split('>>')[::-1])
                retro_rxn = Reaction(retro_smirks)
                if retro_rxn.num_reactant == 2:
                    print("hi")
                all_prods = retro_rxn.rxn.RunReactants((retro_rxn.get_mol(smiles),))      
                succs = list(sk.tree.successors(r))
                if sk.tree.nodes[succs[0]]['child'] == 'right':
                    succs = succs[::-1]
                precursors = [sk.tree.nodes[n]['smiles'] for n in succs]
                iou = 0.
                for prods in all_prods:
       
                    # if len(prods) == 2 and not rxn.is_reactant_first(prods[0]):
                    #     assert rxn.is_reactant_second(prods[0])
                    #     assert rxn.is_reactant_first(prods[1])
                    #     prods = prods[::-1]
                    # else:
                    #     assert rxn.is_reactant_first(prods[0])
                    #     if len(prods) > 1:
                    #         assert rxn.is_reactant_second(prods[1])
    
                    smis = [Chem.MolToSmiles(p) for p in prods]
                    try:
                        prod_fps = [np.array(fp_2048(smi)) for smi in smis]
                        pre_fps = [np.array(fp_2048(smi)) for smi in precursors]
                    except:
                        continue
                    ious = [_tanimoto_similarity(fp1, fp2) for (fp1, fp2) in zip(prod_fps, pre_fps)]
                    iou = max(iou, np.mean(ious))
                if iou < 1:
                    bad_rxn_ids.add(rxn_id)
                    if iou > 0.:
                        pass
    return bad_rxn_ids

sanity_check()




KeyboardInterrupt: 

In [60]:
[fp_2048(precursor) for precursor in precursors]

True

In [ ]:
Pdb.set_trace()